In [2]:
import pandas as pd

df = pd.DataFrame(pd.read_excel("finalmenproducts.xlsx"))

In [3]:
data = df[['description', 'image']]
data.head(2)

,description,image
0,ASOS DESIGN skinny jeans in red croc leather look,images.asos-media.com/products/asos-design-ski...
1,Twisted Tailor Anderson Tall skinny suit set i...,images.asos-media.com/groups/twisted-tailor-an...


In [4]:
image_url = f"http://{data['image'][0]}"
print(image_url)

http://images.asos-media.com/products/asos-design-skinny-jeans-in-red-croc-leather-look/202011410-1-red


In [23]:
# In the dataframe (data), I have the image url as : (images.asos-media.com/products/asos-design-skinny-jeans-in-red-croc-leather-look/202011410-1-red)
# I want to convert it as follow: (http://images.asos-media.com/products/asos-design-skinny-jeans-in-red-croc-leather-look/202011410-1-red)
# Function to add 'http://' to URLs
def add_http(url):
    if url.startswith('http://'):
        return url
    else:
        return 'http://' + url

# Apply the function to the 'image' column
data['image'] = data['image'].apply(add_http)

C:\Users\Najla\AppData\Local\Temp\ipykernel_9356\2295933260.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['image'] = data['image'].apply(add_http)


In [34]:
image_url = data['image'][0]
image_name = os.path.basename(image_url)
local_path = os.path.join(image_dir, image_name)

response = requests.get(image_url)
# print(response.content)
with open(local_path, 'wb') as f:
    f.write(response.content)

In [39]:
import pandas as pd
import requests
from tqdm import tqdm
from PIL import Image
import os

# Assuming you have your original DataFrame 'data' and it contains the 'image' URLs

# Step 1: Download images and save locally
image_dir = "image_data"
os.makedirs(image_dir, exist_ok=True)

for index, row in tqdm(data.iterrows(), total=len(data)-50000):
    image_url = row['image']
    image_name = os.path.basename(image_url)
    local_path = os.path.join(image_dir, image_name)

    response = requests.get(image_url)
    with open(local_path, 'wb') as f:
        f.write(response.content)


 89%|████████▉ | 39858/44705 [3:41:12<26:54,  3.00it/s]   


ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [40]:
# Step 2: Update DataFrame with local paths
data['local_image_path'] = [os.path.join(image_dir, os.path.basename(url)) for url in data['image']]

C:\Users\Najla\AppData\Local\Temp\ipykernel_9356\1049064302.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['local_image_path'] = [os.path.join(image_dir, os.path.basename(url)) for url in data['image']]


In [41]:
data.head(2)

,description,image,local_image_path
0,ASOS DESIGN skinny jeans in red croc leather look,http://images.asos-media.com/products/asos-des...,image_data\202011410-1-red
1,Twisted Tailor Anderson Tall skinny suit set i...,http://images.asos-media.com/groups/twisted-ta...,image_data\68297-group-1


In [100]:
images_captions_dict = {}

# I have a dataframe with 2 columns: description, image I want to convert it to a dectionary
# with image as the key and an array of captions as the value
# Iterate through the DataFrame rows and populate the dictionary
for index, row in data.iterrows():
    image_name = row['local_image_path']
    caption = get_preprocessed_caption(str(row['description']))
    
    if image_name in images_captions_dict:
        images_captions_dict[image_name].append(caption)
    else:
        images_captions_dict[image_name] = [caption]



In [66]:
from PIL import Image
import io
import os
import tensorflow as tf
from tqdm import tqdm

# Assuming you have your image_dataset containing PIL binary images and corresponding paths

# Step 1: Convert and save PIL binary images to JPEG format
image_dir = "converted_images_jpeg"
os.makedirs(image_dir, exist_ok=True)

for index, row in data.iterrows():
    image_name = row['local_image_path']
    pil_image = Image.open(image_name)
    # Convert and save the PIL image as JPEG
    save_path = os.path.join(image_dir, os.path.basename(row['image']))
    pil_image.save(save_path + '.jpeg', format="JPEG")


UnidentifiedImageError: cannot identify image file 'image_data\\9089206-1-grey'

In [98]:
image_dir = '.\converted_images_jpeg'
data['local_image_path'] = [os.path.join(image_dir, os.path.basename(url)) for url in data['image'] + '.jpeg']

C:\Users\Najla\AppData\Local\Temp\ipykernel_9356\2335287032.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['local_image_path'] = [os.path.join(image_dir, os.path.basename(url)) for url in data['image'] + '.jpeg']


In [99]:
data.head(2)

,description,image,local_image_path
0,ASOS DESIGN skinny jeans in red croc leather look,http://images.asos-media.com/products/asos-des...,.\converted_images_jpeg\202011410-1-red.jpeg
1,Twisted Tailor Anderson Tall skinny suit set i...,http://images.asos-media.com/groups/twisted-ta...,.\converted_images_jpeg\68297-group-1.jpeg


In [119]:
data['local_image'] = [os.path.basename(url) for url in data['image'] + '.jpeg']

C:\Users\Najla\AppData\Local\Temp\ipykernel_9356\734247529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['local_image'] = [os.path.basename(url) for url in data['image'] + '.jpeg']


In [120]:
data.head(2)

,description,image,local_image_path,local_image
0,ASOS DESIGN skinny jeans in red croc leather look,http://images.asos-media.com/products/asos-des...,.\converted_images_jpeg\202011410-1-red.jpeg,202011410-1-red.jpeg
1,Twisted Tailor Anderson Tall skinny suit set i...,http://images.asos-media.com/groups/twisted-ta...,.\converted_images_jpeg\68297-group-1.jpeg,68297-group-1.jpeg


In [121]:
print(len(data))

94705


In [124]:
# Define the folder containing the JPEG images
image_folder = "Dataset/Images"

# Assuming you have your DataFrame 'data' with columns 'description', 'image', 'local_image_path', and 'local_image'
# Filter the DataFrame to include only rows with images that exist in the folder
filtered_data = data[data['local_image'].apply(lambda x: os.path.exists(os.path.join(image_folder, x)))]

# Create a new DataFrame with 'local_image' and 'description' columns
result_df = filtered_data[['local_image', 'description']]

# Write the DataFrame to a text file with the desired format
output_file = "Dataset/captions.txt"
result_df.to_csv(output_file, sep=',', index=False)


In [125]:
print(len(result_df))

46344


In [126]:
# Remove the word "ASOS DESIGN" from the 'description' column
result_df['description'] = result_df['description'].str.replace('ASOS DESIGN', '')

C:\Users\Najla\AppData\Local\Temp\ipykernel_9356\618831663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['description'] = result_df['description'].str.replace('ASOS DESIGN', '')


In [128]:
result_df.head(2)

,local_image,description
0,202011410-1-red.jpeg,skinny jeans in red croc leather look
1,68297-group-1.jpeg,Twisted Tailor Anderson Tall skinny suit set i...


In [129]:
output_file = "Dataset/captions.txt"
result_df.to_csv(output_file, sep=',', index=False)